<a href="https://colab.research.google.com/github/AustenLamacraft/learning-wavelets/blob/master/wavelet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
!pip install -q tensorflow-probability
import tensorflow as tf
import tensorflow_probability as tfp
import math
import numpy as np

  ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0-beta1 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0-beta1


In [2]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.

In [3]:
def convolve_circular(a, b):
  '''
  a: vector
  b: kernel
  Requires that 2*tf.size(b) <= tf.size(a). If this is not satisfied, overlap
  will occur in the convolution.
  '''
  b_padding = tf.constant([[0, int(tf.size(a) - tf.size(b))]])
  b_padded = tf.pad(b, b_padding, "CONSTANT")
  a_fft = tf.signal.fft(tf.complex(a, 0.0))
  b_fft = tf.signal.fft(tf.complex(b_padded, 0.0))
  ifft = tf.signal.ifft(a_fft * b_fft)
  return tf.cast(tf.math.real(ifft), 'float32')

In [4]:
class Lifting(tfp.bijectors.Bijector):

  def __init__(self, validate_args=False, name="lifting", n_lifting_coeffs=2):
    super(Lifting, self).__init__(
        validate_args=validate_args,
        forward_min_event_ndims=1,
        name=name)
    self.n_lifting_coeffs = n_lifting_coeffs
    self.s_coeff = tf.Variable(initial_value=tf.random.uniform(shape=(n_lifting_coeffs,)))  
    self.t_coeff = tf.Variable(initial_value=tf.random.uniform(shape=(n_lifting_coeffs,)))
    self.K = tf.Variable(initial_value=1, dtype='float32')

  def _forward(self, x):
    x_evens = x[0::2]
    x_odds = x[1::2]
    odds_conv_s = convolve_circular(x_odds, self.s_coeff)
    ulp = x_evens - odds_conv_s  # ulp: unnormalized lowpass
    ulp_conv_t = convolve_circular(ulp, self.t_coeff)
    ubp = x_evens - ulp_conv_t  # ubp: unnormalized bandpass
    lp = ulp * tf.broadcast_to(1/self.K, ulp.shape)
    bp = ubp * tf.broadcast_to(self.K, ubp.shape)
    return tf.stack([lp, bp])

  def _inverse(self, y):
    # TODO
    pass

  def _inverse_log_det_jacobian(self, y):
    return 0

  def _forward_log_det_jacobian(self, x):
    return 0

In [7]:
lifting = Lifting()

x = tf.constant([1,2,3,4,5,6,7,8], dtype='float32')

lifting._forward(x)

<tf.Tensor: id=272, shape=(2, 4), dtype=float32, numpy=
array([[-1.9137051,  1.2909219,  2.142216 ,  2.9935102],
       [ 1.8808206,  2.3854785,  3.1196704,  4.307269 ]], dtype=float32)>